In [15]:
import pandas as pd

In [16]:
import axelrod as axl
import numpy as np

In [3]:
import imp
prepare_data = imp.load_source('prepare_data', '../src/prepare_data.py')
process_data = imp.load_source('processe_data', '../src/process_data.py')

In [22]:
data = '../data/probend_noise.csv'

In [23]:
df = pd.read_csv(data, index_col=0)

/home/nightwing/anaconda3/envs/axlml/lib/python3.5/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [24]:
df.columns

Index(['Rank', 'Name', 'Median_score', 'Cooperation_rating', 'Wins',
       'Initial_C_rate', 'CC_rate', 'CD_rate', 'DC_rate', 'DD_rate',
       'CC_to_C_rate', 'CD_to_C_rate', 'DC_to_C_rate', 'DD_to_C_rate', 'noise',
       'probend', 'repetitions', 'size', 'seed'],
      dtype='object')

In [25]:
noise = pd.read_csv('../data/noise_processed.csv', index_col=0)

/home/nightwing/anaconda3/envs/axlml/lib/python3.5/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [27]:
noise.columns

Index(['Rank', 'Name', 'Median_score', 'Cooperation_rating', 'Wins',
       'Initial_C_rate', 'CC_rate', 'CD_rate', 'DC_rate', 'DD_rate',
       'CC_to_C_rate', 'CD_to_C_rate', 'DC_to_C_rate', 'DD_to_C_rate', 'noise',
       'turns', 'repetitions', 'size', 'seed', 'SSeeror', 'Normalized_Rank',
       'Stochastic', 'Memory_depth', 'Makes_use_of_game',
       'Makes_use_of_length', 'Memory_usage', 'Cooperation_rating_max',
       'Cooperation_rating_min', 'Cooperation_rating_median',
       'Cooperation_rating_mean', 'Cooperation_rating_comp_to_max',
       'Cooperation_rating_comp_to_min', 'Cooperation_rating_comp_to_median',
       'Cooperation_rating_comp_to_mean'],
      dtype='object')

In [28]:
prob = pd.read_csv('../data/probend_processed.csv', index_col=0)

/home/nightwing/anaconda3/envs/axlml/lib/python3.5/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [29]:
prob.columns

Index(['Rank', 'Name', 'Median_score', 'Cooperation_rating', 'Wins',
       'Initial_C_rate', 'CC_rate', 'CD_rate', 'DC_rate', 'DD_rate',
       'CC_to_C_rate', 'CD_to_C_rate', 'DC_to_C_rate', 'DD_to_C_rate',
       'probend', 'repetitions', 'size', 'seed', 'SSeeror', 'Normalized_Rank',
       'Stochastic', 'Memory_depth', 'Makes_use_of_game',
       'Makes_use_of_length', 'Memory_usage', 'Cooperation_rating_max',
       'Cooperation_rating_min', 'Cooperation_rating_median',
       'Cooperation_rating_mean', 'Cooperation_rating_comp_to_max',
       'Cooperation_rating_comp_to_min', 'Cooperation_rating_comp_to_median',
       'Cooperation_rating_comp_to_mean'],
      dtype='object')

**STRATEGY HAS BEEN REMOVE BECAUSE IT WAS A DUPLICATE**

In [34]:
replace = {'Slow Tit For Two Tats': "Tit For 2 Tats"}

In [35]:
df = df.replace(replace)

**Extra columns**

- SSE error
- Strategies features

**sse error**

In [36]:
def get_error_for_row(row):
    columns_for_sse = [
        "CC_to_C_rate",
        "CD_to_C_rate",
        "DC_to_C_rate",
        "DD_to_C_rate",
    ]
    vector = [row[column] for column in columns_for_sse]
    sse_error = prepare_data.get_least_squares(vector=vector)

    return sse_error


def get_normalised_rank(row):
    return row["Rank"] / (row["size"] - 1)

def fix_name(row):
    return row['Name'].split(':')[0]

In [37]:
%time df['SSeeror'] = df.apply(get_error_for_row, axis=1)

CPU times: user 2min 36s, sys: 1min 31s, total: 4min 7s
Wall time: 2min 4s


In [38]:
%time df['Normalized_Rank'] = df.apply(get_normalised_rank, axis=1)

CPU times: user 31.4 s, sys: 511 ms, total: 32 s
Wall time: 32 s


In [39]:
%time df['Name'] = df.apply(fix_name, axis=1)

CPU times: user 20.1 s, sys: 572 ms, total: 20.7 s
Wall time: 20.7 s


**Strategies properties**

In [40]:
strategies_properties = process_data.get_strategies_properties()

In [41]:
strategies_properties = strategies_properties.set_index('Name')

In [42]:
df = pd.merge(df, strategies_properties, left_on='Name', right_index=True, how='left', sort=False);

In [43]:
len(df)

1098915

In [44]:
len(df)

1098915

In [45]:
df.isnull().values.any()

False

In [46]:
def get_memory_percentage(row):
    if np.isinf(row['Memory_depth']):
        return 1
    if row['Memory_depth'] == 0:
        return 0
    return row['Memory_depth'] / row['size']

In [47]:
%time df['Memory_usage'] = df.apply(get_memory_percentage, axis=1)

CPU times: user 33.6 s, sys: 484 ms, total: 34.1 s
Wall time: 34.1 s


In [48]:
df.isnull().values.any()

False

In [49]:
max_coop = pd.DataFrame(df.groupby('seed')['Cooperation_rating'].max())

In [50]:
df = pd.merge(df, max_coop, left_on='seed', right_index=True, how='left', sort=False);

In [51]:
df.to_csv(output)

**Cleaning data**

In [52]:
to_drop = df[df['Normalized_Rank'] > 1]['seed'].unique()

In [53]:
to_drop

array([], dtype=float64)

In [54]:
df = df[~(df['seed'].isin(to_drop))]

In [26]:
df[df['Normalized_Rank'] > 1]['seed'].unique()

array([], dtype=float64)